<a href="https://colab.research.google.com/github/jumbokh/python_learn/blob/master/notebooks/RAG%E6%96%87%E4%BB%B6%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Retrieval Augmented Generation (RAG) App
> 建置 RAG 問答應用程式

## 什麼是 RAG？  
RAG（Retrieval Augmented Generation）是一種將語言模型（LLM）的生成能力與檢索系統結合的技術。透過先從外部知識庫中檢索相關文件，再將檢索到的內容當作 prompt 的一部分，能讓模型回答私有資料或訓練截止後的新資訊相關問題。若模型不確定答案，就應直接回覆「我不知道」以確保回應真實可靠。

## RAG 核心概念  
- **索引（Indexing）**  
  1. **載入（Load）**：使用 Document Loader 將資料來源（例如 PDF、Markdown 同檔案）讀取成 `Document` 物件。  
  2. **切分（Split）**：以 Text Splitter 將大型文件切成可被模型處理的小段落，避免超過上下文視窗限制。  
  3. **存儲（Store）**：將切分後的段落利用 Embeddings 轉成向量，並儲存在 VectorStore（例如 Chroma、FAISS）中以便後續檢索。

![](https://python.langchain.com/v0.2/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

- **檢索與生成（Retrieval & Generation）**  
  1. **檢索（Retrieve）**：用 Retriever 根據使用者提問搜尋最相關的向量段落。  
  2. **生成（Generate）**：將檢索到的段落與使用者問題一起送入 ChatModel（或 LLM），讓模型生成回答。

![](https://python.langchain.com/v0.2/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

本教學將示範如何利用 LangChain 與 LLM 服務（例如免費的 Groq 方案）來打造一個能夠學習特定領域知識並回應使用者問題的問答應用。主要步驟包含以下幾個部分：

1. **環境安裝與依賴設定**
2. **註冊免費 LLM 服務（Groq）**
3. **建立 LLM 物件**
4. **基礎問答展示**
5. **匯入並處理特定領域的 PDF 文件（Load）**
6. **文件切割（Split）**
7. **向量化存儲：建立向量庫（Store）**
8. **檢索（Retrieval）**
9. **生成答案（Generation）**

## 1. 環境安裝與依賴設定

首先，透過 pip 指令安裝所有需要的依賴包，包括 LangChain 各模組、PDF 讀取工具以及 HuggingFace 模型包。你可以在 Jupyter Notebook 中執行下列指令：


In [1]:
!pip install langchain langchain_community langchain_chroma langchain-openai pypdf langchain_huggingface -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.2 MB/s eta 0:00:0

In [ ]:
import warnings
warnings.filterwarnings('ignore') # 關閉一些提示訊息讓畫面更乾淨(不影響程式運行結果)

此指令將安裝 LangChain 核心庫以及用於文件載入、向量資料庫（Chroma）和 LLM 呼叫等相關套件。

## 2. 註冊免費 LLM 服務（Groq）

如果你尚未擁有付費的 OpenAI API，可使用免費資源，例如 [Groq](https://console.groq.com/login)。Groq 提供類似 Chat Completions 的 API 端點，LangChain 的 `ChatOpenAI` 也能整合其他業者的模型。此外，除了 Groq，還可以使用 GitHub Models、Google Gemini、Ollama 等服務。

參考下面連結以進一步了解 Groq 的使用：
- [Groq 使用教學](https://andy6804tw.github.io/crazyai-llm/free-llm-api-integration-resources/groq-tutorial/)

取得 API Key 後，將 API 金鑰儲存起來：

In [ ]:
GROQ_API_KEY = 'Your-Key' # 請自行填入

## 3. 建立 LLM 物件

使用 LangChain 的 `ChatOpenAI` 並指定模型與 API 端點來調用不同服務。以下範例示範如何使用 Groq 提供的 API（以 "qwen-qwq-32b" 模型為例）：

In [ ]:
from langchain_openai import ChatOpenAI
from google.colab import userdata

llm = ChatOpenAI(model="qwen-qwq-32b",openai_api_base="https://api.groq.com/openai/v1", openai_api_key=GROQ_API_KEY)

這樣配置後，你即可利用 `llm` 物件來發送訊息，並根據指定提示生成回答。

## 4. 基礎問答展示

以下示例中，建立一個系統提示（system prompt）與問答訊息，並呼叫 LLM：

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

system_prompt = (
    """
    你是個問答助理。
    如果你不知道答案，就回覆「我不知道」。
    回答最多三句，並保持精簡。
    請使用繁體中文回答問題。
    """
)

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="請問「人工智慧學程」活動規劃安排幾場演講?"),
]

print(llm.invoke(messages).content)


<think>
好的，我需要回答用户关于“人工智慧學程”活动规划安排的演讲场次问题。首先，我应该回忆之前是否有相关的信息被提供过，或者是否有公开的资料可以参考。

根据用户的问题，他们询问的是演讲的具体场数。但在我现有的数据中，没有明确提到这个活动的详细安排。可能之前的对话或提供的资料中没有具体的数字，因此我无法确定准确的数字。

另外，根据用户的指示，如果不知道答案，必须回答“我不知道”，并且要保持回答简短。因此，我需要确认是否有任何遗漏的信息来源，或者是否存在其他途径可以获取这个数据。如果确实没有，最安全的做法就是按照指示回复“我不知道”。

可能还需要考虑是否有其他相关活动或常见的演讲场次作为参考，但这样可能会导致不准确的答案，所以为了避免误导用户，应该坚持不猜测，直接说明不知道。

总结起来，正确的做法是告知用户自己不知道具体场次，并建议他们查阅官方信息或联系主办方获取准确信息。这样既符合用户的指示，也避免提供错误信息。
</think>

我不知道。建議確認活動官方宣發資料或聯繫主辦單位。


In [ ]:
# 如果想模擬 ChatGPT 那樣串流輸出可以用 stream()
for token in llm.stream(messages):
    print(token.content, end="", flush=True)


<think>
好的，我需要回答用户关于“人工智慧學程”活动规划中有多少场演讲的问题。首先，我要回忆一下之前是否有收到过相关的信息或者数据。根据我所知，用户的问题可能涉及活动的具体安排细节。不过，我需要确认自己是否真的知道这个信息。

如果我没有具体的数据，正确的做法应该是直接回复“我不知道”，而不是编造数字。根据用户提供的指示，如果不知道答案，必须回答“我不知道”，并且回答要简洁，最多三句话。因此，我需要先检查自己的知识库是否有相关信息。

假设我的知识库中没有关于这个活动的具体演讲场次的信息，那么正确的回应就是直接告知用户不知道。这样可以避免提供错误的信息，保持回答的准确性和可靠性。用户可能希望得到确切的数字，但如果没有确切答案，诚实是最好的选择。

另外，用户可能需要进一步的帮助，但根据问题要求，不需要提供额外信息，只需简短回答。因此，我应该严格遵守指示，只回复“我不知道”即可，不需要扩展其他内容。
</think>

我不知道。

由於我提問了一個 AI 也不清楚的東西，因為我沒有提供我的知識文件。這裡定義的 prompt 告知 LLM 若不確定答案則回覆「不知道」。你可以藉由修改提示模板調整回應風格與細節。

## 5. 匯入並處理特定領域的 PDF 文件（Load）

假設我們有一些 PDF 文件，內容涵蓋特定領域的知識，本節示範如何以這些文件作為資料來源，讓 LLM 根據內文回答問題。

### 5.1 查看及下載 PDF 文件

首先，請手動上傳自己的 PDF 文件，以下程式檢查目前目錄中是否已有 PDF 文件：

In [ ]:
import os

for filename in os.listdir():
  if filename.endswith(".pdf"):
    print(filename)

人工智慧學程活動規劃.pdf


### 5.2 載入 PDF 文件

使用 `PyPDFLoader` 載入 PDF 文件，提取文件內容及 metadata：

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

all_docs = []
for filename in os.listdir():
    if filename.endswith(".pdf"):
        print("載入檔案:", filename)
        loader = PyPDFLoader(filename)
        docs = loader.load()
        print("提取文件:", docs)
        all_docs.extend(docs)
        print()

載入檔案: 人工智慧學程活動規劃.pdf


提取文件: [Document(metadata={'producer': 'macOS Version 15.1.1 (Build 24B91) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250419150628Z00'00'", 'moddate': "D:20250419150628Z00'00'", 'source': '人工智慧學程活動規劃.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='「人工智慧學程」活動規劃 活動緣由 近年來 生成式 AI（Generative AI）、強化學習（RL） 與 可解釋 AI（XAI） 等技術蓬勃發展，已成為推動企業數位轉型與創新服務的關鍵引擎。然而，許多校園與企業對「如何將最新 AI 技術快速落地」仍在摸索。為縮短理論與實務的落差，並強化跨域人才的即戰力，我們規劃了本系列「演講與活動時程」，核心目的如下： 1. 聚焦前沿主題 • 從生成式 AI 在行銷場景的應用起步，依序涵蓋多模態學習、LangChain + RAG 實作、Edge AI 部署，再到 AI 倫理治理，內容緊扣產業趨勢 & 技術脈動。 • 透過漸進式主題安排，讓學員循序體驗「概念 → 技術 → 實作 → 應用 → 法規」的完整拼圖。 2. 橋接學界與業界 • 邀請新創創辦人、一線研發工程師與資深顧問現身分享實戰經驗，降低「紙上談兵」風險。 • 親赴企業參訪，近距離觀摩 Edge AI 部署與模型治理流程，打破課堂與產線的隔閡。 3. 強化跨域協作力 • 多元活動形式（演講、工作坊、論壇、企業參訪），鼓勵學員在不同情境動手實作、討論，培養問題拆解與團隊合作能力。 • 工作坊聚焦 RAG 與 Edge AI MVP 打造，讓學員於短時間內完成「資料檢索 → 模型部署」全流程驗證。 4. 落實倫理與永續思維 • 年末「AI 倫理與治理論壇」帶領學員正視偏見、資安與隱私議題，學習制定風險控管與永續策略，接軌國際法規趨勢。 感謝國科會計畫支持，透過本系列規劃，我們期望培養兼具 技術深度 × 產業廣度 × 倫理高度 的未來人才，讓參與者不僅能掌握 AI 技術，亦能在職涯與實務場域中發揮影響力，成為驅動

In [ ]:
# 檢查總文件頁數
print("總文件頁數:", len(all_docs))

總文件頁數: 6


In [ ]:
# 查看文件部分內容
print("第一頁內容前 100 字：", all_docs[0].page_content[0:100])

第一頁內容前 100 字： 「人工智慧學程」活動規劃 活動緣由 近年來 生成式 AI（Generative AI）、強化學習（RL） 與 可解釋 AI（XAI） 等技術蓬勃發展，已成為推動企業數位轉型與創新服務的關鍵引擎。然而，


## 6. 文件切割（Split）

由於單個 PDF 頁面可能過長，我們需要將每份文件切割成較小的片段以便後續嵌入與檢索。使用 `RecursiveCharacterTextSplitter` 並設定 `chunk_size` 與 `chunk_overlap`：


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

all_splits = text_splitter.split_documents(all_docs)
print(f"文件總共切成 {len(all_splits)} 等分")

文件總共切成 6 等分


In [ ]:
# 觀察切割後每個 chunk 資訊
for i in range(len(all_splits)):
  print(all_splits[i].metadata)

{'producer': 'macOS Version 15.1.1 (Build 24B91) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250419150628Z00'00'", 'moddate': "D:20250419150628Z00'00'", 'source': '人工智慧學程活動規劃.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1', 'start_index': 0}
{'producer': 'macOS Version 15.1.1 (Build 24B91) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250419150628Z00'00'", 'moddate': "D:20250419150628Z00'00'", 'source': '人工智慧學程活動規劃.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2', 'start_index': 0}
{'producer': 'macOS Version 15.1.1 (Build 24B91) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250419150628Z00'00'", 'moddate': "D:20250419150628Z00'00'", 'source': '人工智慧學程活動規劃.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3', 'start_index': 0}
{'producer': 'macOS Version 15.1.1 (Build 24B91) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250419150628Z00'00'", 'moddate': "D:20250419150628Z00'00'", 'source': '人工智慧學程活動規劃.pdf', 'total_pag

## 7. 向量化存儲：建立向量庫（Store）

將所有文檔片段轉換成向量，再存入向量資料庫，以方便後續檢索。本範例使用 HuggingFace 嵌入模型與 Chroma 向量庫。

### 7.1 轉換為嵌入向量 (Embedding)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

### 7.2 建立向量庫
這裡使用 LangChain 的 Chroma 向量資料庫，把文件切片（chunks）轉成向量並且儲存起來，方便後續做相似度檢索。

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# vector_store = Chroma.from_documents(documents=all_splits, embedding=embeddings)

## 8. 檢索（Retrieval）

建立檢索器，根據使用者查詢從向量庫中檢索最相關的文檔片段。設定檢索參數（例如使用 cosine 相似度，並回傳前 k 個結果）：


In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})
retrieved_docs = retriever.invoke("七月有幾場演講？")
print("檢索到的文件數量:", len(retrieved_docs))

# 印出檢索文件的 metadata 以確認結果：
for i in range(len(retrieved_docs)):
    print("檢索結果", i, ":", retrieved_docs[i].metadata)

檢索到的文件數量: 2
檢索結果 0 : {'creationdate': "D:20250419150628Z00'00'", 'creator': 'PyPDF', 'moddate': "D:20250419150628Z00'00'", 'page': 1, 'page_label': '2', 'producer': 'macOS Version 15.1.1 (Build 24B91) Quartz PDFContext', 'source': '人工智慧學程活動規劃.pdf', 'start_index': 0, 'total_pages': 6}
檢索結果 1 : {'creationdate': "D:20250419150628Z00'00'", 'creator': 'PyPDF', 'moddate': "D:20250419150628Z00'00'", 'page': 2, 'page_label': '3', 'producer': 'macOS Version 15.1.1 (Build 24B91) Quartz PDFContext', 'source': '人工智慧學程活動規劃.pdf', 'start_index': 0, 'total_pages': 6}


In [ ]:
# 嘗試其他查詢：
retrieved_docs = retriever.invoke("論壇將由時主持？")
for i in range(len(retrieved_docs)):
    print("檢索結果", i, ":", retrieved_docs[i].metadata)

檢索結果 0 : {'creationdate': "D:20250419150628Z00'00'", 'creator': 'PyPDF', 'moddate': "D:20250419150628Z00'00'", 'page': 5, 'page_label': '6', 'producer': 'macOS Version 15.1.1 (Build 24B91) Quartz PDFContext', 'source': '人工智慧學程活動規劃.pdf', 'start_index': 0, 'total_pages': 6}
檢索結果 1 : {'creationdate': "D:20250419150628Z00'00'", 'creator': 'PyPDF', 'moddate': "D:20250419150628Z00'00'", 'page': 1, 'page_label': '2', 'producer': 'macOS Version 15.1.1 (Build 24B91) Quartz PDFContext', 'source': '人工智慧學程活動規劃.pdf', 'start_index': 0, 'total_pages': 6}


## 9. 生成答案（Generation）

將檢索到的文檔（context）與使用者問題結合，利用預先定義的 prompt 模板傳入 LLM，生成最終答案。以下提供幾種實現方式：

### 9.1 使用 Hub 中的 RAG Prompt

首先，從 LangChain Hub 下載現成的 [RAG prompt](https://smith.langchain.com/hub/rlm/rag-prompt)：

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "OpenBI 是一間台灣的新創公司，專門做生物醫療。", "question": "請問OpenBI是什麼？"}
).to_messages()

print("Prompt 範例：\n", example_messages[0].content)

Prompt 範例：
 You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: 請問OpenBI是什麼？ 
Context: OpenBI 是一間台灣的新創公司，專門做生物醫療。 
Answer:


利用 LangChain Expression Language (LCEL) Runnable protocol 的「|」運算子，把各個步驟串在一起。

- `RunnablePassthrough()`：把輸入的「問題」原封不動傳遞給後續元件。
- `StrOutputParser()`：從 LLM 回傳的 ChatResult 裡取出最終的文字回答。


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# 定義格式化函式將檢索到的文檔內容組合成字符串
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 整合 retrieval 與生成的完整流程（chain）建立
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 使用 stream 模式逐步印出生成回答
for chunk in rag_chain.stream("七月有幾場演講？"):
    print(chunk, end="", flush=True)


<think>
Okay, the user is asking how many lectures there are in July. Let me look at the provided context. The first part lists events from July to September. The July entries are on July 8th and July 22nd. Both are labeled as "演講" (lecture). Then in August, there's a workshop, and September has an enterprise visit. The second part is for October-December, which isn't relevant here. So counting the July lectures: two entries. Wait, need to make sure there's no other events in July. The first section has weeks 1, 3, 6, 9. Weeks 1 and 3 are July 8 and 22, which are both July. Week 6 is August 12, so those are the only two lectures in July. The others are workshops or different months. So the answer should be 2.
</think>

七月有兩場演講。分別安排在7月8日和7月22日，主題為「生成式AI在數位行銷的應用」及「多模態學習入⾨」。 其他月份的活動不計入。

### 9.2 使用自訂義的提示樣板
就像上面展示的那樣，我們可以從 LangChain 的 Prompt Hub 載入內建的提示模板（像是 [RAG prompt](https://smith.langchain.com/hub/rlm/rag-prompt)），但其實你也可以很簡單地客製化自己的提示格式，讓模型依照你的需求調整回應風格。

舉個例子，假如你希望回答要更有條列式結構，而且最後用中文跟使用者說謝謝，就可以像下面這樣修改原本的模板內容：

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """根據以下提供的資料來回答問題，請務必根據上下文內容作答。
如果不知道答案，就誠實說不知道，不要亂猜。
請用條列式的方式列出重點，最多不超過三點，並用繁體中文回答。
最後請在結尾說「謝謝您的提問！」。

{context}

問題：{question}

回答（繁體中文）："""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("張芷瑄的背景資訊？"))


<think>
好的，我需要回答關於張芷瑄的背景資訊。先看問題，要求根據提供的資料，用條列式列出最多三點，且用繁體中文。如果不確定就說不知道。

首先，我會回到資料中的「演講1」部分，找到她的講者簡介。她目前的職位是台灣某知名數位媒體集團AI創新部的副理。她的專長是將大規模語言模型（LLM）應用在內容生成、受眾洞察及成效預測。

接下來，她主導過「智慧選題推薦系統」專案，使得編輯團隊產能提升35%。這個成就是她的實際經驗之一，應作為第二點。

第三點，她曾多次受邀到AdTech Taiwan分享生成式AI與廣告科技的整合實例。這顯示她的外部活動和專業認可，所以第三點這裡。

要確保沒有超過三點，並且每點都是關鍵簡介。其他資訊如演講內容或論壇部分不相關，不需要提及。沒有更多關於她的教育背景或其他職務的資料，所以只能列出這三點。

最後檢查是否符合要求：條列式、繁體、不超過三點，且只使用提供的資訊。沒問題，這樣應該對了。
</think>

1. 現任台灣知名數位媒體集團 AI 創新部副理，專精於將大規模語言模型（LLM）應用於內容生成、受眾洞察及成效預測。  
2. 曾主導「智慧選題推薦系統」專案，成功提升編輯團隊產能達35%，具實務專案 leadership 經驗。  
3. 多次受邀至AdTech Taiwan活動分享生成式 AI 與廣告科技整合案例，具業界專業影響力。  

謝謝您的提問！


### 9.3 LangChain 進階寫法

到目前為止我們已經完成RAG練習，也可以利用 LangChain 提供的高階函式建立問答。LangChain 提供兩個方便的 RAG 內建 chain 函式，讓你不用自己手動串接每個 runnable，就能快速建構 Retrieval‑Augmented Generation 應用：

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
你是一位幫助使用者解答問題的小助手，請依照下方提供的檢索內容進行回答。
如果你無法從內容中找到答案，請誠實說不知道，不要亂猜。
請使用繁體中文作答，最多三句話，簡潔扼要。

{context}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "請問企業參訪有幾間以及公司名稱？"})
print(response["answer"])


<think>
好的，我现在要回答用户的问题：“請問企業參訪有幾間以及公司名稱？”首先，我需要仔细查看提供的资料，找到相关的部分。

根据用户提供的资料，企业參訪部分在“企業參訪1: Edge AI 部署實務”这一章节。里面提到導覽團隊是XYZ公司智慧製造事業處。接下来，我需要确认是否有其他企業參訪的条目。比如是否有企業參訪2，但查看内容后，发现资料中只提到了企業參訪1，没有其他企業參訪的条目。

因此，可以确定只有企業參訪1，而其中的公司是XYZ公司。用户的问题是问企業參訪的間数和名称，所以答案应该是1間，XYZ公司。需要确认是否有其他隐藏的信息或可能的遗漏。比如是否有在論壇或者其他部分提到的公司，但是根据提供的资料，其他部分如論壇提到的公司如台積電和Google属于论坛的与谈嘉宾，并非企業參訪的公司。所以答案正确。
</think>

企業參訪共1間，公司名稱為XYZ公司。


## Reference
- [Build a Retrieval Augmented Generation (RAG) App](https://python.langchain.com/v0.2/docs/tutorials/rag/)